In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.dates as mdates
import datetime
from collections import defaultdict, OrderedDict
import random
import re
import json
from datetime import datetime, timedelta
import arrow

import sys, os

# Lancie

In [2]:
data_path = os.path.join('lancie', 'CallGraphEdge.csv')

In [3]:
df = pd.read_csv(data_path, sep='\t', header=None).loc[:, [1, 3]]
df.columns = ['caller', 'callee']
df

,caller,callee
0,<java.io.File: java.lang.String[] list()>/java...,<java.io.UnixFileSystem: java.lang.String[] li...
1,<sun.util.calendar.ZoneInfoFile: void <clinit>...,<java.security.AccessController: java.lang.Obj...
2,<sun.util.calendar.ZoneInfoFile: void <clinit>...,<java.security.AccessController: java.lang.Obj...
3,<java.lang.invoke.InnerClassLambdaMetafactory:...,<java.security.AccessController: java.lang.Obj...
4,<sun.security.util.AbstractAlgorithmConstraint...,<java.security.AccessController: java.lang.Obj...
...,...,...
57920,<ch.wisv.areafiftylan.products.controller.Orde...,<org.springframework.web.servlet.support.Servl...
57921,<ch.wisv.areafiftylan.products.controller.Tick...,<net.logstash.logback.marker.Markers: net.logs...
57922,<ch.wisv.areafiftylan.products.controller.Orde...,<net.logstash.logback.marker.Markers: net.logs...
57923,<ch.wisv.areafiftylan.products.controller.Orde...,<net.logstash.logback.marker.Markers: net.logs...


## Caller & Callee

1. If a callee has "Team", then its caller definately have "team"

In [4]:
df_both = df[(df.caller.str.contains("Team") & df.callee.str.contains("Team"))]
df_both

,caller,callee
3953,<ch.wisv.areafiftylan.teams.service.TeamServic...,<ch.wisv.areafiftylan.teams.service.TeamServic...
3954,<ch.wisv.areafiftylan.teams.controller.TeamRes...,<ch.wisv.areafiftylan.teams.service.TeamServic...
4192,<ch.wisv.areafiftylan.utils.ExportController: ...,<ch.wisv.areafiftylan.teams.model.TeamExportDT...
4307,<ch.wisv.areafiftylan.teams.service.TeamServic...,<ch.wisv.areafiftylan.teams.model.TeamInviteRe...
4530,<ch.wisv.areafiftylan.teams.controller.TeamRes...,<ch.wisv.areafiftylan.teams.model.TeamDTO: jav...
...,...,...
46315,<ch.wisv.areafiftylan.utils.ExportController: ...,<ch.wisv.areafiftylan.teams.service.TeamServic...
46316,<ch.wisv.areafiftylan.teams.controller.TeamRes...,<ch.wisv.areafiftylan.teams.service.TeamServic...
57613,<ch.wisv.areafiftylan.teams.model.Team: boolea...,<ch.wisv.areafiftylan.teams.model.Team: int ge...
57614,<ch.wisv.areafiftylan.teams.model.Team: int ha...,<ch.wisv.areafiftylan.teams.model.Team: int ge...


## Call tree

In [5]:
parent2children = defaultdict(list) # caller:[a list of callees]
child2parents = defaultdict(list) # callee:[a list of callers]
n = len(df_both)

for i in range(n):
    caller, callee = df_both.iloc[i, :]
    caller = caller.split('/')[0][1:-1]
    callee = callee[1:-1]
    parent2children[caller].append(callee)
    child2parents[callee].append(caller)

def printTree(dic):
    for key, val in dic.items():
        print(key)
        print(val)
        print("*****************************************************************************")

print(len(parent2children))
printTree(parent2children)

35
ch.wisv.areafiftylan.teams.service.TeamServiceImpl: boolean removeMember(java.lang.Long,java.lang.String)
['ch.wisv.areafiftylan.teams.service.TeamServiceImpl: ch.wisv.areafiftylan.teams.model.Team delete(java.lang.Long)', 'ch.wisv.areafiftylan.teams.service.TeamServiceImpl: ch.wisv.areafiftylan.teams.model.Team getTeamById(java.lang.Long)']
*****************************************************************************
ch.wisv.areafiftylan.teams.controller.TeamRestController: org.springframework.http.ResponseEntity removeTeamMember(java.lang.Long,java.lang.String)
['ch.wisv.areafiftylan.teams.service.TeamServiceImpl: boolean removeMember(java.lang.Long,java.lang.String)']
*****************************************************************************
ch.wisv.areafiftylan.utils.ExportController: ch.wisv.areafiftylan.teams.model.TeamExportDTO lambda$exportUsers$2(ch.wisv.areafiftylan.teams.model.Team)
['ch.wisv.areafiftylan.teams.model.TeamExportDTO: void <init>(java.lang.Long,java.lang.

In [6]:
print(len(child2parents))
printTree(child2parents)

35
ch.wisv.areafiftylan.teams.service.TeamServiceImpl: ch.wisv.areafiftylan.teams.model.Team delete(java.lang.Long)
['ch.wisv.areafiftylan.teams.service.TeamServiceImpl: boolean removeMember(java.lang.Long,java.lang.String)']
*****************************************************************************
ch.wisv.areafiftylan.teams.service.TeamServiceImpl: boolean removeMember(java.lang.Long,java.lang.String)
['ch.wisv.areafiftylan.teams.controller.TeamRestController: org.springframework.http.ResponseEntity removeTeamMember(java.lang.Long,java.lang.String)']
*****************************************************************************
ch.wisv.areafiftylan.teams.model.TeamExportDTO: void <init>(java.lang.Long,java.lang.String,java.lang.Long,java.util.List)
['ch.wisv.areafiftylan.utils.ExportController: ch.wisv.areafiftylan.teams.model.TeamExportDTO lambda$exportUsers$2(ch.wisv.areafiftylan.teams.model.Team)']
*****************************************************************************
ch.

In [7]:
def findRootsOrLeaves(graph_dic):
    res = set()
    statusMap = defaultdict(int)
    
    def dfs(node):
        if statusMap[node] != 0:
            if statusMap[node] == 1:
                print("recursive on %s!" % node )
            return
        
        statusMap[node] = 1
        if node not in graph_dic:
            res.add(node)
        else:
            for adjacent in graph_dic[node]:
                dfs(adjacent)
        statusMap[node] = 2
        
    for node in list(graph_dic.keys()):
        dfs(node)
    return res

roots = findRootsOrLeaves(child2parents)
leaves = findRootsOrLeaves(parent2children)
print("Root number: %d" % len(roots))
for root in roots:
    print(root)

print("\n**************************************************")
print("Leaves number: %d" % len(leaves))
for leaf in leaves:
    print(leaf)

roots & leaves # Empty - roots and leaves not intersect.

Root number: 26
ch.wisv.areafiftylan.users.controller.CurrentUserRestController: java.util.List getOpenInvites(ch.wisv.areafiftylan.users.model.User)
ch.wisv.areafiftylan.utils.mail.MailRestController: org.springframework.http.ResponseEntity sendMailToTeam(java.lang.Long,ch.wisv.areafiftylan.utils.mail.MailDTO)
ch.wisv.areafiftylan.utils.ExportController: ch.wisv.areafiftylan.teams.model.TeamExportDTO lambda$exportUsers$2(ch.wisv.areafiftylan.teams.model.Team)
ch.wisv.areafiftylan.teams.controller.TeamRestController: org.springframework.http.ResponseEntity add(ch.wisv.areafiftylan.users.model.User,ch.wisv.areafiftylan.teams.model.TeamDTO)
ch.wisv.areafiftylan.teams.controller.TeamRestController: org.springframework.http.ResponseEntity inviteTeamMember(java.lang.Long,java.lang.String)
ch.wisv.areafiftylan.teams.controller.TeamRestController: org.springframework.http.ResponseEntity acceptTeamInvite(java.lang.String)
ch.wisv.areafiftylan.teams.controller.TeamRestController: ch.wisv.areafi

set()

### Notice
1. Figured out that the "caller" column includes the "callee"
2. One caller may call multiple callee
3. Security sensitive functions not included in call grah
    1. TeamRepository.saveAndFlush(...)
    2. Teamrepository.delete(...)